# Data Preprocessing

### Step 1: Set up

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# notebook configurations
pd.options.display.max_colwidth = 1000

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:

from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("drive/MyDrive/COGS 109 Amazon Project/Data/amazon_products_sampled_raw.csv")
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
0,BELWABA Small Fabric Sling Bag Side Bag Handmade For Women,accessories,Handbags & Clutches,https://m.media-amazon.com/images/W/IMAGERENDERING_521856-T2/images/I/61E1IPfgdiL._AC_UL320_.jpg,https://www.amazon.in/BELWABA-Womens-Fabric-Handmade-Sling/dp/B09V81TRPT/ref=sr_1_17201?qid=1679159167&s=shoes&sr=1-17201,5.0,4,₹999,"₹2,999"
1,Wesley 15 Liter Casual Daypack/collegeDaypack/tuitionbag/Backpack,accessories,Bags & Luggage,https://m.media-amazon.com/images/W/IMAGERENDERING_521856-T2/images/I/81CGoQxYIxL._AC_UL320_.jpg,https://www.amazon.in/Wesley-Daypack-collegeDaypack-tuitionbag-Backpack/dp/B0BVVY98PZ/ref=sr_1_7585?qid=1679144145&s=luggage&sr=1-7585,NaN,NaN,₹198,₹699
2,LMDPRAJAPATIS PRAJAPATI GEMS Certified Unheated Untreatet 6.25 Ratti 5.75 Carat Natural Ruby Loose Gemstone For Women's an...,accessories,Jewellery,https://m.media-amazon.com/images/W/IMAGERENDERING_521856-T2/images/I/41-JVuZj3LL._AC_UL320_.jpg,https://www.amazon.in/PRAJAPATI-GEMS-LMDPRAJAPATIS-Certified-Untreated/dp/B07NCY81B8/ref=sr_1_1822?qid=1679145212&s=jewelry&sr=1-1822,3.4,171,₹579,"₹3,899"
3,Handcuffs Side Sling Cross Body Messenger Bag for Men Small Size (Tan),accessories,Bags & Luggage,https://m.media-amazon.com/images/W/IMAGERENDERING_521856-T1/images/I/71CsC5XCXGL._AC_UL320_.jpg,https://www.amazon.in/Handcuffs-Trendy-Color-Sling-BFSLNG30/dp/B079T41BXY/ref=sr_1_4064?qid=1679144023&s=luggage&sr=1-4064,4.2,"1,348",₹649,₹999
4,Baggit LXE STRIPY2 E PHEEBY BLACK,accessories,Bags & Luggage,https://m.media-amazon.com/images/I/51a+gMcx+kL._AC_UL320_.jpg,https://www.amazon.in/LXE-STRIPY2-E-PHEEBY-BLACK/dp/B09MTP8JS5/ref=sr_1_2398?qid=1679143967&s=luggage&sr=1-2398,4.3,91,₹695,"₹1,390"
...,...,...,...,...,...,...,...,...,...
77179,Grey block heels for women,women's shoes,Fashion Sandals,https://m.media-amazon.com/images/W/IMAGERENDERING_521856-T2/images/I/31PmOf546VL._AC_UL320_.jpg,https://www.amazon.in/Grey-block-heels-women-numeric_9/dp/B09ZV6932C/ref=sr_1_18908?qid=1679211827&s=shoes&sr=1-18908,NaN,NaN,₹700,"₹1,500"
77180,Carlton London Womens Cll-6584 Ballet Flat,women's shoes,Ballerinas,https://m.media-amazon.com/images/W/IMAGERENDERING_521856-T1/images/I/61FswmH8QeL._AC_UL320_.jpg,https://www.amazon.in/Carlton-London-CLL-6584/dp/B0B2JRLXBV/ref=sr_1_5855?qid=1679211885&s=shoes&sr=1-5855,NaN,NaN,₹598,"₹1,495"
77181,"BATA Womens Mary Heels, (7616700)",women's shoes,Shoes,https://m.media-amazon.com/images/W/IMAGERENDERING_521856-T1/images/I/71dZVU2C2KL._AC_UL320_.jpg,https://www.amazon.in/Bata-Womens-Mary-Heels-7616700/dp/B0B153T1LZ/ref=sr_1_224?qid=1679211497&s=shoes&sr=1-224,4.1,159,₹701,"₹1,299"
77182,Denim Expos Traditional Bellies UK-Size-8 White | DNXP-MS-56,women's shoes,Ballerinas,https://m.media-amazon.com/images/I/81SbPqBR2qL._AC_UL320_.jpg,https://www.amazon.in/Denim-Traditional-Bellies-UK-Size-8-DNXP-MS-56/dp/B08PQ4MSJ1/ref=sr_1_11278?qid=1679211926&s=shoes&sr=1-11278,NaN,NaN,NaN,NaN


In [3]:
df.columns

Index(['name', 'main_category', 'sub_category', 'image', 'link', 'ratings',
       'no_of_ratings', 'discount_price', 'actual_price'],
      dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77184 entries, 0 to 77183
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            77184 non-null  object
 1   main_category   77184 non-null  object
 2   sub_category    77184 non-null  object
 3   image           77184 non-null  object
 4   link            77184 non-null  object
 5   ratings         54447 non-null  object
 6   no_of_ratings   54447 non-null  object
 7   discount_price  67033 non-null  object
 8   actual_price    75063 non-null  object
dtypes: object(9)
memory usage: 5.3+ MB


In [5]:
df["main_category"].unique()

array(['accessories', 'appliances', 'bags & luggage', 'beauty & health',
       'car & motorbike', 'grocery & gourmet foods', 'home & kitchen',
       'home, kitchen, pets', 'industrial supplies', "kids' fashion",
       "men's clothing", "men's shoes", 'music', 'pet supplies',
       'sports & fitness', 'stores', 'toys & baby products',
       'tv, audio & cameras', "women's clothing", "women's shoes"],
      dtype=object)

### Step 2: Subset the data

Our original dataset contains columns that are not needed for our analysis, as such we'll subset the data accordingly.

In [6]:
df = df[["name", "main_category"]]

### Step 3: Check for nulls and/or empty rows

In [7]:
name_null_or_empty_rows = df[df["name"].isnull() | (df["name"] == "")]
name_null_or_empty_rows

,name,main_category


In [8]:
main_category_null_or_empty_rows = df[df["main_category"].isnull() | (df["main_category"] == "")]
main_category_null_or_empty_rows

,name,main_category


### Step 4: Remove punctuation

Removing punctuation in data cleaning is important for noise reduction, text standardization, tokenization, and improving language syntax. Punctuation marks usually do not carry significant meaning on their own and can introduce unnecessary noise in the text data. By removing punctuation, the text is standardized, making it easier for the model to process and analyze. Additionally, punctuation marks are typically treated as separate tokens during tokenization, and removing them helps create cleaner and more meaningful tokens.

In [9]:
df_cleaned = df

In [10]:
df_cleaned["name"] = df_cleaned["name"].apply(lambda x: re.sub(r"[^a-zA-Z0-9\s]", " ", str(x)))

In [11]:
df_cleaned[["name"]]

,name
0,BELWABA Small Fabric Sling Bag Side Bag Handmade For Women
1,Wesley 15 Liter Casual Daypack collegeDaypack tuitionbag Backpack
2,LMDPRAJAPATIS PRAJAPATI GEMS Certified Unheated Untreatet 6 25 Ratti 5 75 Carat Natural Ruby Loose Gemstone For Women s an
3,Handcuffs Side Sling Cross Body Messenger Bag for Men Small Size Tan
4,Baggit LXE STRIPY2 E PHEEBY BLACK
...,...
77179,Grey block heels for women
77180,Carlton London Womens Cll 6584 Ballet Flat
77181,BATA Womens Mary Heels 7616700
77182,Denim Expos Traditional Bellies UK Size 8 White DNXP MS 56


### Step 5: Remove words with number

Removing words with numbers during data cleaning of product names eliminates numerical information that may not be relevant for the task at hand. Numerical values in product names often represent specific attributes, such as sizes or model numbers, which may not contribute to product understanding or classification. By removing these words, we focus on descriptive and discriminative features, simplifying the text representation for accurate predictions.

In [12]:
# Remove words containing a number (brand numbers, product id, etc.)
df_cleaned["name"] = df_cleaned["name"].apply(lambda x: ' '.join([word for word in x.split() if not re.search(r'\d', word)]))
df_cleaned[["name"]]

,name
0,BELWABA Small Fabric Sling Bag Side Bag Handmade For Women
1,Wesley Liter Casual Daypack collegeDaypack tuitionbag Backpack
2,LMDPRAJAPATIS PRAJAPATI GEMS Certified Unheated Untreatet Ratti Carat Natural Ruby Loose Gemstone For Women s an
3,Handcuffs Side Sling Cross Body Messenger Bag for Men Small Size Tan
4,Baggit LXE E PHEEBY BLACK
...,...
77179,Grey block heels for women
77180,Carlton London Womens Cll Ballet Flat
77181,BATA Womens Mary Heels
77182,Denim Expos Traditional Bellies UK Size White DNXP MS


### Step 6: Remove single string characters

Removing single-character strings during data cleaning is important as they often do not provide meaningful information and can introduce noise. By removing them, we can improve the quality of the text data and focus on more relevant words. This helps to reduce dimensionality, eliminate unnecessary noise, and improve the efficiency of subsequent text analysis tasks.

In [13]:
df_cleaned["name"] = df_cleaned["name"].str.replace(r'\b\w\b', "", regex = True)
df_cleaned[["name"]]

,name
0,BELWABA Small Fabric Sling Bag Side Bag Handmade For Women
1,Wesley Liter Casual Daypack collegeDaypack tuitionbag Backpack
2,LMDPRAJAPATIS PRAJAPATI GEMS Certified Unheated Untreatet Ratti Carat Natural Ruby Loose Gemstone For Women an
3,Handcuffs Side Sling Cross Body Messenger Bag for Men Small Size Tan
4,Baggit LXE PHEEBY BLACK
...,...
77179,Grey block heels for women
77180,Carlton London Womens Cll Ballet Flat
77181,BATA Womens Mary Heels
77182,Denim Expos Traditional Bellies UK Size White DNXP MS


### Step 7: Lowercase

Lowercasing text during data cleaning in our project is important for standardization and consistency. It treats words with different cases as the same, reducing the vocabulary size and improving performance in subsequent tasks. Lowercasing ensures that words like ```Hello``` and ```hello``` are represented uniformly, making it easier to compare and analyze the text data.

In [14]:
df_cleaned["name"] = df_cleaned["name"].str.lower()
df_cleaned

,name,main_category
0,belwaba small fabric sling bag side bag handmade for women,accessories
1,wesley liter casual daypack collegedaypack tuitionbag backpack,accessories
2,lmdprajapatis prajapati gems certified unheated untreatet ratti carat natural ruby loose gemstone for women an,accessories
3,handcuffs side sling cross body messenger bag for men small size tan,accessories
4,baggit lxe pheeby black,accessories
...,...,...
77179,grey block heels for women,women's shoes
77180,carlton london womens cll ballet flat,women's shoes
77181,bata womens mary heels,women's shoes
77182,denim expos traditional bellies uk size white dnxp ms,women's shoes


### Step 8: Remove stop words

Stop words are commonly used words such as "a," "the," "is," which do not carry significant meaning and can introduce noise to the analysis. By removing stop words, we can reduce the dimensionality of the data and focus on more informative and content-rich words.

In [15]:
def remove_stop_words(input_string):
    # Tokenize the string
    tokens = word_tokenize(input_string)

    # Get the list of English stop words
    stop_words = set(stopwords.words("english"))

    # Remove stop words
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]

    # Reconstruct the string
    output_string = " ".join(filtered_tokens)

    return output_string

In [16]:
# Apply to column "name"
df_cleaned['name'] = df_cleaned['name'].apply(remove_stop_words)

In [17]:
df_cleaned.name

0                                                         belwaba small fabric sling bag side bag handmade women
1                                                 wesley liter casual daypack collegedaypack tuitionbag backpack
2        lmdprajapatis prajapati gems certified unheated untreatet ratti carat natural ruby loose gemstone women
3                                               handcuffs side sling cross body messenger bag men small size tan
4                                                                                        baggit lxe pheeby black
                                                          ...                                                   
77179                                                                                     grey block heels women
77180                                                                      carlton london womens cll ballet flat
77181                                                                                     bata w

In [18]:
df_cleaned[["name"]]

,name
0,belwaba small fabric sling bag side bag handmade women
1,wesley liter casual daypack collegedaypack tuitionbag backpack
2,lmdprajapatis prajapati gems certified unheated untreatet ratti carat natural ruby loose gemstone women
3,handcuffs side sling cross body messenger bag men small size tan
4,baggit lxe pheeby black
...,...
77179,grey block heels women
77180,carlton london womens cll ballet flat
77181,bata womens mary heels
77182,denim expos traditional bellies uk size white dnxp ms


### Step 9: Part-of-Speech (POS) Tagging and Lemmatize the words

POS tagging identifies the grammatical category of each word, while lemmatization reduces words to their base form. By using POS tags, we accurately lemmatize words, ensuring consistent representation across different grammatical forms.

In [19]:
def get_wordnet_pos(tag):
    if tag.startswith("N"):
        return wordnet.NOUN
    elif tag.startswith("V"):
        return wordnet.VERB
    elif tag.startswith("J"):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [20]:
lemmatizer = WordNetLemmatizer()
sent = "kids smart watch for girls toy for kids gift for girls watches"

tagged_words = nltk.pos_tag(word_tokenize(sent))
for word, w_tag in tagged_words:
    lemma_tag = get_wordnet_pos(w_tag)
    lemma = lemmatizer.lemmatize(word, lemma_tag)
    print(f"{word} ({w_tag}) --> {lemma} ({lemma_tag})")

kids (NNS) --> kid (n)
smart (VBP) --> smart (v)
watch (NN) --> watch (n)
for (IN) --> for (n)
girls (NNS) --> girl (n)
toy (NN) --> toy (n)
for (IN) --> for (n)
kids (NNS) --> kid (n)
gift (NN) --> gift (n)
for (IN) --> for (n)
girls (NNS) --> girl (n)
watches (NNS) --> watch (n)


In [21]:
df_cleaned["name"] = df_cleaned["name"].apply(lambda x: " ".join([lemmatizer.lemmatize(word, pos = get_wordnet_pos(w_tag)) for word, w_tag in nltk.pos_tag(word_tokenize(x))]))

In [22]:
df_cleaned[["name"]]

,name
0,belwaba small fabric sling bag side bag handmade woman
1,wesley liter casual daypack collegedaypack tuitionbag backpack
2,lmdprajapatis prajapati gem certify unheated untreatet ratti carat natural ruby loose gemstone woman
3,handcuff side sling cross body messenger bag men small size tan
4,baggit lxe pheeby black
...,...
77179,grey block heel woman
77180,carlton london womens cll ballet flat
77181,bata woman mary heel
77182,denim expos traditional belly uk size white dnxp m


### Step 10: Data Validation

#### Check for nulls or empty rows. Drop rows it it exists

**Note**: Remember majority of our categories have an equal amount of instances. Dropping the rows below will slightly decrease the number of instances for their respective categories

In [23]:
# Strip whitespace and replace other representations of empty values
df_cleaned["name"] = df_cleaned["name"].str.strip().replace("", np.nan)

In [24]:
name_null_or_empty_rows = df_cleaned[df_cleaned["name"].isnull() | (df_cleaned["name"] == "")]
name_null_or_empty_rows

,name,main_category
3728,NaN,accessories
64881,NaN,"tv, audio & cameras"
76534,NaN,women's shoes


In [25]:
df_cleaned[df_cleaned["name"].str.strip().isnull()]

,name,main_category
3728,NaN,accessories
64881,NaN,"tv, audio & cameras"
76534,NaN,women's shoes


In [26]:
df_cleaned.drop(name_null_or_empty_rows.index, inplace=True)

In [27]:
df_cleaned.dropna(subset=["name"], inplace=True)

In [28]:
name_null_or_empty_rows = df_cleaned[df_cleaned["name"].isnull() | (df_cleaned["name"] == "")]
name_null_or_empty_rows

,name,main_category


In [29]:
# this will obviously be 0 since it was 0 in the raw. just double checking
main_category_null_or_empty_rows = df_cleaned[df_cleaned["main_category"].isnull() | (df_cleaned["main_category"] == "")]
main_category_null_or_empty_rows

,name,main_category


### Step 11: Export cleaned DF

In [30]:
df_cleaned

,name,main_category
0,belwaba small fabric sling bag side bag handmade woman,accessories
1,wesley liter casual daypack collegedaypack tuitionbag backpack,accessories
2,lmdprajapatis prajapati gem certify unheated untreatet ratti carat natural ruby loose gemstone woman,accessories
3,handcuff side sling cross body messenger bag men small size tan,accessories
4,baggit lxe pheeby black,accessories
...,...,...
77179,grey block heel woman,women's shoes
77180,carlton london womens cll ballet flat,women's shoes
77181,bata woman mary heel,women's shoes
77182,denim expos traditional belly uk size white dnxp m,women's shoes


In [31]:
df_cleaned.to_csv("drive/MyDrive/COGS 109 Amazon Project/Data/amazon_products_sampled_cleaned.csv", index = False)